<p>Similar to relational databases, the Spark DataFrame and Dataset APIs
and Spark SQL offer a series of join transformations: inner joins, outer joins, left
joins, right joins, etc. All of these operations trigger a large amount of data movement
across Spark executors.</p>

<h3>Five join strategiy</h3>
<ol>
    <li>The broadcast hash join (BHJ)</li>
    <li>Shuffle hahs join (SHJ)</li>
    <li>Shuffle sort merge join (SMJ)</li>
    <li>Broadcast nested loop join (BNLJ)</li>
    <li>Shuffle and replicated nested loop join (Castesian product)</li>
</ol>

<p> By default Spark will use a <strong>broadcast join</strong> if the smaller data set is less than 10 MB. <br>
This configuration is set in spark.sql.autoBroadcastJoinThreshold; <br>
# By setting this value to -1 broadcasting can be disabled. <br>
    We use <strong>Sort Merge Joins if we are joining two big tables.</strong></p>


In [1]:
! ls -l ~/datasets

total 473232
-rw-rw-r--. 1 train train  42658497 Nov  3 22:07 201508_trip_data.csv
-rw-rw-r--. 1 train train      4556 Jul 21 18:58 Advertising.csv
drwxrwxr-x. 2 train train      4096 Nov 10 22:02 cat_images
-rw-rw-r--. 1 train train    674857 Sep 17 22:49 Churn_Modelling.csv
drwxrwxr-x. 3 train train        96 Oct  6 12:18 churn-telecom
-rw-rw-r--. 1 train train  41002480 Oct  6 12:18 Fire_Incidents.csv.gz
drwxrwxr-x. 7 train train        67 Oct  6 12:18 flight-data
-rw-rw-r--. 1 train train  46401315 Oct  6 12:18 Hotel_Reviews.csv.gz
drwxrwxr-x. 2 train train       198 Sep 17 20:53 hotel_reviews_parquet
-rw-rw-r--. 1 train train       180 Sep  2 11:40 insanlar.csv
-rw-rw-r--. 1 train train      4611 Sep  1 16:13 iris.csv
-rw-rw-r--. 1 train train     15802 Sep  1 16:13 iris.json
-rw-rw-r--. 1 train train    325145 Sep  1 16:14 kuruyemisler.txt
-rw-rw-r--. 1 train train  44525776 Oct  6 12:18 market1mil.csv.gz
drwxrwxr-x. 2 train train       198 Nov 10 22:17 market1mil_snappyparquet
-

In [2]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = (
SparkSession.builder.appName("Joins").master("local[2]")
    .config("spark.executor.memory","3g")
    .config("spark.driver.memory","512m")
    .config("spark.memory.fraction","0.1")
    .config("spark.memory.storageFraction","0.0")
    .getOrCreate()
)

In [4]:
# If you don't anything against it the small df lt 10 MB will is broadcasted
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

## Shuffle Sort Merge Join

In [15]:
prevented_bcast_join_df = categories.join(departments, 
                                categories.categoryDepartmentId == departments.departmentId)

In [16]:
prevented_bcast_join_df.explain()

== Physical Plan ==
*(5) SortMergeJoin [categoryDepartmentId#17], [departmentId#54], Inner
:- *(2) Sort [categoryDepartmentId#17 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(categoryDepartmentId#17, 200), true, [id=#166]
:     +- *(1) Project [categoryId#16, categoryDepartmentId#17, categoryName#18]
:        +- *(1) Filter isnotnull(categoryDepartmentId#17)
:           +- FileScan csv [categoryId#16,categoryDepartmentId#17,categoryName#18] Batched: false, DataFilters: [isnotnull(categoryDepartmentId#17)], Format: CSV, Location: InMemoryFileIndex[file:/home/train/venvspark/dev/data/retail_db/categories.csv.gz], PartitionFilters: [], PushedFilters: [IsNotNull(categoryDepartmentId)], ReadSchema: struct<categoryId:int,categoryDepartmentId:int,categoryName:string>
+- *(4) Sort [departmentId#54 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(departmentId#54, 200), true, [id=#175]
      +- *(3) Project [departmentId#54, departmentName#55]
         +- *(3) Filter isnot

In [21]:
spark.stop()